# Data Skeptic Release Automation

* Final asset
* Upload to libsyn
* Enter libsyn details
* Schedule in Lisbyn
* Run update-content.ipynb
* verify site
* facebook post
* transcription

* remove ad (mike north; fiver)
* upload to special place
* regenerate member feed


In [1]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
import urllib.request
from mutagen.mp3 import MP3

In [2]:
album_art = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg'

In [3]:
config = json.load(open('../config.json', 'r'))

In [4]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [5]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)

In [6]:
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [10]:
fname = 'feed.xml'

In [11]:
if os.path.isfile(fname):
    os.remove(fname)

In [12]:
url = 'http://dataskeptic.com/feed.rss'
url = 'http://dataskeptic.libsyn.com/rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

fetching


In [13]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [14]:
len(episodes)

233

In [15]:
guid = episodes[0]['guid']['#text']
print(episodes[0]['title'], guid)

Deep Fakes 8ed692d61b7e49a3a587800b57fda923


In [13]:
r = requests.get('https://dataskeptic.com/api/refresh')
r.content.decode('utf-8')

'{"status":"ok"}'

In [15]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
LIMIT 10
"""
df = pd.read_sql(q, conn)

In [16]:
if df.iloc[0]['guid'] == '':
    print("ERROR: missing GUID!")
    print("Going to use guid=" + guid)
    if not(df.iloc[0]['guid'] in emap):
        q2 = "update blog set guid='{guid}' where blog_id={blog_id}".format(guid=guid, blog_id = df.iloc[0]['blog_id'])
        print(q2)
        conn.execute(q2)

df = pd.read_sql(q, conn)
print(df.shape[0], 'result:', df.iloc[0]['title'], df.iloc[0]['guid'])

ERROR: missing GUID!
Going to use guid=ec43d43fb2464988ab8f72f6a56f0262
update blog set guid='ec43d43fb2464988ab8f72f6a56f0262' where blog_id=409
1 result: DeepFakes ec43d43fb2464988ab8f72f6a56f0262


In [17]:
#blog_id=404
#q = "update blog set guid='{guid}' where blog_id={blog_id}".format(guid=guid, blog_id=blog_id)
#print(q)
#conn.execute(q)

In [18]:
tpl = """
INSERT INTO related_content (blog_id, dest, type, title, body, duration) VALUES 
({blog_id}, '{dest}', '{type}', '{title}', '{body}', {duration})
"""

In [19]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title'].replace("'", "\\'")
    body = row['abstract'].replace("'", "\\'")
    if guid in emap:
        print("Adding")
        dest = emap[guid]
        response = urllib.request.urlopen(dest)
        data = response.read()
        fname = 'temp.mp3'
        f = open(fname, 'wb')
        f.write(data)
        f.close()
        audio = MP3(fname)
        duration = int(audio.info.length)
        q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
        r1 = conn.execute(q1)
        dest = album_art
        q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
        r2 = conn.execute(q2)

Adding


In [16]:
conn.execute("UPDATE blog SET guid='8ed692d61b7e49a3a587800b57fda923' WHERE blog_id=409")

In [17]:
pd.read_sql("select blog_id, prettyname, title, publish_date, guid from blog order by blog_id desc limit 10", conn)

,blog_id,prettyname,title,publish_date,guid
0,409,/episodes/2018/deepfakes,DeepFakes,2018-09-21 00:00:00,8ed692d61b7e49a3a587800b57fda923
1,408,/meta/2018/data-skeptic-internship,Data Skeptic Internship,2018-09-18 00:00:00,
2,407,/episodes/2018/fake-news-midterm,Fake News Midterm,2018-09-14 00:00:00,ec43d43fb2464988ab8f72f6a56f0262
3,406,/episodes/2018/quality-score,Quality Score,2018-09-06 00:00:00,0705f1fc3e274304a96938ab99f11bd8
4,405,/transcripts/2018/the-knowledge-illusion,The Knowledge Illusion,2018-08-29 00:00:00,
5,404,/episodes/2018/the-knowledge-illusion,The Knowledge Illusion,2018-08-31 00:00:00,fd038ece71fa49bcaff8de17dcfdf3ed
6,403,/episodes/2018/click-through-rates,Click Through Rates,2018-08-23 00:00:00,fabf4476db954cec97f403328f729bc9
7,402,/episodes/2018/algorithmic-detection-of-fake-news,Algorithmic Detection of Fake News,2018-08-17 15:14:58,626d7d33cb734db583b1c57734417e86
8,401,/episodes/2018/ant-intelligence,Ant Intelligence,2018-08-10 00:00:00,2f286792dfab4e5a89320c006076b24b
9,400,/episodes/2018/human-detection-of-fake-news,Human Detection of Fake News,2018-08-02 00:00:00,0771428607a843ee943f841b822d1f2f


In [25]:
#conn.execute("update blog set abstract='The scale and frequency with which information can be distributed on social media makes the problem of fake news a rapidly metastasizing issue. To do any content filtering or labeling demands an algorithmic solution.  \nIn today''s episode, Kyle interviews Kai Shu and Mike Tamir about their independent work exploring the use of machine learning to detect fake news.' where blog_id=402")

# Bonus feed stuff

In [23]:
pd.read_sql("SELECT * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,709,400,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Human Detection of Fake News,Human Detection of Fake News,2018-08-03 15:04:53,NaN,-1.0
1,708,400,http://traffic.libsyn.com/dataskeptic/human-de...,mp3,Human Detection of Fake News,Human Detection of Fake News,2018-08-03 15:04:53,NaN,1706.0
2,707,400,,internal-link,Detecting Pseudo-profound BS,Our first interview with Gordon Pennycook,2018-08-03 02:43:53,93.0,NaN
3,706,400,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Gordon Pennycook,Gordon Pennycook is an assistant professor of ...,2018-08-03 02:42:42,-1.0,NaN
4,705,399,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Spam Filtering with Naive Bayes,Today's spam filters are advanced data driven ...,2018-07-27 15:28:20,NaN,-1.0
5,704,399,http://traffic.libsyn.com/dataskeptic/spam-fil...,mp3,Spam Filtering with Naive Bayes,Today's spam filters are advanced data driven ...,2018-07-27 15:28:20,NaN,1184.0
6,703,398,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,,,2018-07-24 03:18:08,NaN,NaN
7,702,398,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Robert Sheaffer,Robert Sheaffer is a freelance writer and skep...,2018-07-24 03:15:45,NaN,NaN
8,700,398,https://s3.amazonaws.com/data-skeptic-bonus-fe...,mp3,Robert Sheaffer,,2018-07-24 03:04:54,-1.0,1740.0
9,699,397,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,The Spread of Fake News,How does fake news get spread online? Its not ...,2018-07-20 16:40:36,NaN,-1.0


In [24]:
pd.read_sql("SELECT dest from related_content where content_id=696", conn).iloc[0]['dest']

'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg'

In [16]:
pd.read_sql("SELECT * from bonus_episodes", conn)

,episode_id,title,link,img,desc,pubDate,enclosure_url,guid,last_updated,blog_id
0,1,Introduction,https://dataskeptic.com,https://s3.amazonaws.com/data-skeptic-bonus-fe...,So what's the deal with the bonus feed? How d...,2017-05-01 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,0b79795d3efc95b9976c7c5b933afce2,2017-05-08 01:24:59,NaN
1,2,"Jill Darling, Survey Director for the Understa...",https://dataskeptic.com/blog/polling/2017/inte...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Featured recently on Data Skeptic, this is the...",2017-05-02 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,7453e2daa86d4e028125f4ee74be97c5,2018-04-01 01:06:14,252.0
2,3,Easily Fooling Deep Neural Networks,https://dataskeptic.com/blog/episodes/2015/eas...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,This episode is from the Data Skeptic archives...,2017-05-12 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,67c7eec88871451c9603d3b361b51e49,2017-05-12 23:42:28,NaN
3,4,"Ernie Tedeschi, Economics, on Poll Re-Weighting",https://dataskeptic.com/blog/polling/2017/inte...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,Our recent episode Opinion Polling for Preside...,2017-05-07 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,4d67d8112ac54a85bea403d842937d37,2018-04-01 01:06:19,254.0
4,5,"David Carmona, GM of AI at Microsoft",https://dataskeptic.com/blog/tools-and-techniq...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Discussion of AI, cognitive services, and brin...",2017-06-15 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,59105f0a4ee011e7be430e0f2d4e9582,2018-04-01 01:05:43,267.0
5,6,"Rohan Kumar, GM for the Database Systems Group...",https://dataskeptic.com/blog/tools-and-techniq...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Discussion of database as a service, database ...",2017-06-12 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,5bb2dacb4ee011e7be430e0f2d4e9582,2018-04-01 01:05:42,266.0
6,7,Prophecy Fulfillment,https://dataskeptic.com/blog/bonus/prophecy-fu...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,A short recording made in Vietnam,2017-09-12 03:46:50,https://s3.amazonaws.com/data-skeptic-bonus-fe...,02a58e0c976d11e7999d0e0f2d4e9582,2018-04-01 01:03:30,288.0
7,8,Deploying Machine Learning to Production with ...,https://dataskeptic.com/blog/bonus/2018/deploy...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"In this episode, I discuss the process with To...",2018-01-29 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,5d2718172a354a85beac03d84e93ffab,2018-04-01 01:03:29,333.0
8,10,AI at Microsoft,https://dataskeptic.com/blog/bonus/2018/ai-at-...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,In this episode I catch up with Paige Bailey a...,2018-03-27 14:38:12,https://s3.amazonaws.com/data-skeptic-bonus-fe...,234518172a654affff6c03684e6332f2,2018-04-01 01:03:26,357.0
9,11,Charlie Maloney - Journalist and Loebner Prize...,https://dataskeptic.com/blog/bonus/2018/charli...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,This is the uncut version of my interview with...,2018-04-13 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,8172ef3ffa24a8abea714335c07d85da,2018-04-13 21:25:05,364.0


In [13]:
pd.read_sql("""
SELECT t2.guid, t2.title, t1.content_id, t1.blog_id, t1.duration, t3.guid
from related_content t1
JOIN blog t2
 on t1.blog_id = t2.blog_id
LEFT JOIN bonus_episodes t3
 on t1.dest = t3.enclosure_url
WHERE t1.type='mp3'
AND dest like '%%amazonaws%%'
AND (t2.guid is null or t2.guid='' or t2.guid='None')
AND t3.guid is not null
""", conn)

,guid,title,content_id,blog_id,duration,guid


In [11]:
conn.execute("""
UPDATE related_content t1, blog t2, bonus_episodes t3
SET t2.guid = t3.guid
WHERE t1.blog_id = t2.blog_id
 AND t1.dest = t3.enclosure_url
AND t1.type='mp3'
AND dest like '%%amazonaws%%'
AND (t2.guid is null or t2.guid='' or t2.guid='None')
AND t3.guid is not null
""")

In [14]:
pd.read_sql("SELECT * from blog order by blog_id desc limit 10", conn)

,blog_id,prettyname,title,author,abstract,date_created,publish_date,last_rendered,src_file,c_hash,guid,member_link,last_indexed
0,398,/bonus/2018/robert-sheaffer,Robert Sheaffer,Kyle,"In this episode, Kyle interviews skeptical aut...",2018-07-24 02:31:17,2018-07-23 00:00:00,2018-07-24 02:31:17,bonus/2018/robert-shaeffer.htm,8bac26fc5ee56bf4ff7b94f595df137e,123ba8a724353a81be1ac03d84e93ff5a,,2018-07-24 02:35:02
1,397,/episodes/2018/the-spread-of-fake-news,The Spread of Fake News,Kyle,How does fake news get spread online? Its not ...,2018-07-20 05:54:38,2018-07-20 17:27:17,2018-07-20 16:50:57,episodes/2018/the-spread-of-fake-news.htm,2acc9b4c75f876e2b55aba9b8190e945,d089f82bbe174f1b99d711b28f5c9ebb,,2018-07-20 16:55:02
2,396,/episodes/2018/fake-news,Fake News,Kyle,"This episode kicks off our new theme of ""Fake ...",2018-07-13 02:51:28,2018-07-13 15:20:54,2018-07-13 03:56:16,episodes/2018/fake-news.htm,575fe7c2decbc7d54a02b09251519fec,5b239f1adacc4d51b265d2bcaf572c31,,2018-07-13 04:00:03
3,395,/episodes/2018/dev-ops-for-data-science,Dev Ops for Data Science,Kyle,We revisit the 2018 Microsoft Build in this ep...,2018-07-11 20:10:18,2018-07-11 20:26:06,2018-07-11 20:22:36,episodes/2018/dev-ops-for-data-science.htm,22edb694572bf5d72201521d3f8da41b,db35a82146574f52ab5e8650d1bb9f43,,2018-07-11 20:25:02
4,394,/episodes/2018/first-order-logic,First Order Logic,Kyle,Logic is a fundamental of mathematical systems...,2018-07-06 04:38:08,2018-07-06 15:23:39,2018-07-06 21:32:36,episodes/2018/first-order-logic.htm,39a007daafc687da076f00cdab6f4d66,df74ad6688b241ce8ff152f0e4b926b0,,2018-07-06 21:35:02
5,393,/episodes/2018/blind-spots-in-reinforcement-le...,Discovering Blind Spots in Reinforcement Learning,Kyle,An intelligent agent trained in a simulated en...,2018-06-29 05:36:12,2018-06-29 15:32:15,2018-06-29 05:36:12,episodes/2018/blind-spots-in-reinforcement-lea...,597368225536bc73177fdfad7cb34f57,a8feaf4664e54744b24e9950b9da50a5,,2018-06-29 05:45:02
6,392,/episodes/2018/transfer-learning,Transfer Learning,Kyle,"On a long car ride, Linhda and Kyle record a s...",2018-06-15 06:42:11,2018-06-15 15:23:39,2018-06-15 06:42:11,episodes/2018/transfer-learning.htm,c2cba9ed3e0b2ede1620db126c38f805,8fa5993bd9554b068f33566eeddb325c,,2018-06-15 06:50:02
7,391,/episodes/2018/medical-imaging-training-techni...,Medical Imaging Training Techniques,Kyle,Medical imaging is a highly effective tool use...,2018-06-08 03:06:42,2018-06-08 14:06:04,2018-06-08 06:40:24,episodes/2018/medical-imaging-training-techniq...,fc85d859a2158e2f044a736d9ba7da00,4b89f3615aa94de8b994abee770ac011,,2018-06-08 06:45:03
8,390,/episodes/2018/defending-against-adversarial-a...,Defending Against Adversarial Attacks,Kyle,"In this weeks episode, our host Kyle intervie...",2018-06-07 16:09:07,2018-06-22 15:25:18,2018-06-22 03:38:50,episodes/2018/defending-against-adversarial-at...,6cda3be3d5ace5f7e68fbc15e50cd924,b4f3fa58280246e59fd16759f90905c7,,2018-06-22 05:45:02
9,389,/episodes/2018/kalman-filters,Kalman Filters,Kyle,A Kalman Filter is a technique for taking a se...,2018-06-01 06:57:39,2018-06-01 14:02:38,2018-06-01 07:34:02,episodes/2018/kalman-filters.htm,efa80798e87a741d3f5fbacc57333794,cde6f37bec7b4d4889f4c099c549fd90,,2018-06-01 07:35:02


In [15]:
conn.execute("UPDATE bonus_episodes set blog_id=398 WHERE episode_id=24")

In [18]:
conn.execute("update related_content set duration=29*60 where content_id=700")

In [20]:
conn.execute("""insert into related_content 
             (blog_id, dest, type, title, body)
             values 
             (398, 
             'https://s3.amazonaws.com/dataskeptic.com/guests/2018/robert-sheaffer.jpg'
             , 'person', 'Robert Sheaffer', 'Robert Sheaffer is a freelance writer and skeptical investigator. His work appears in publications such as Skeptical Inquirer and Fate Magazine. He was a founding member of the Center For Inquiries''s UFO Subcommittee. He is the author of five books.')""")

In [25]:
conn.execute("""insert into related_content 
             (blog_id, dest, type, title, body)
             values 
             (398, 
             'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg',
             'homepage-image', '', '')""")

In [26]:
pd.read_sql("select * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,703,398,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,,,2018-07-24 03:18:08,NaN,NaN
1,702,398,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Robert Sheaffer,Robert Sheaffer is a freelance writer and skep...,2018-07-24 03:15:45,NaN,NaN
2,700,398,https://s3.amazonaws.com/data-skeptic-bonus-fe...,mp3,Robert Sheaffer,,2018-07-24 03:04:54,-1.0,1740.0
3,699,397,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,The Spread of Fake News,How does fake news get spread online? Its not ...,2018-07-20 16:40:36,NaN,-1.0
4,698,397,http://traffic.libsyn.com/dataskeptic/the-spre...,mp3,The Spread of Fake News,How does fake news get spread online? Its not ...,2018-07-20 16:40:36,NaN,2717.0
5,697,397,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Filippo Menczer,Filippo Menczer is a professor of informatics ...,2018-07-20 06:48:22,-1.0,NaN
6,696,396,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Fake News,"This episode kicks off our new theme of ""Fake ...",2018-07-13 15:21:40,NaN,-1.0
7,695,396,http://traffic.libsyn.com/dataskeptic/fake-new...,mp3,Fake News,"This episode kicks off our new theme of ""Fake ...",2018-07-13 15:21:39,NaN,2298.0
8,694,396,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Robert Sheaffer,Robert Sheaffer is a freelance writer and skep...,2018-07-13 03:36:16,-1.0,NaN
9,693,396,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Brad Schwartz,"Brad Schwartz is the author of ""Broadcast Hyst...",2018-07-13 03:33:58,-1.0,NaN


In [22]:
conn.execute("update related_content set dest='https://s3.amazonaws.com/dataskeptic.com/guests/2018/robert-sheaffer.jpg' where content_id=702")